### **Physics-Informed Neural Networks and Symbolic Regression**
https://arxiv.org/pdf/2307.08107.pdf

#### **Caso de estudio**

- du1 / dt = e^(-t/10) * u2 * u3
- du2 / dt = u1 * u3
- du3 / dt = -2 * u1 * u2

Con condiciones iniciales:

- u1(0) = 1
- u2(0) = 0.8
- u3(0) = 0.5

Transformamos las ED:

- du1 / dt = f1(t, u1, u2, u3)
- du2 / dt = f2(t, u1, u2, u3)
- du3 / dt = a * u1 * u2 + b

Donde f1 y f2 son funciones desconocidas, y a y b son los parametros desconocidos.

**PINN for System Identification**

In [1]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt 
from scipy.integrate import solve_ivp

import tensorflow as tf  
from tensorflow import keras
tf.random.set_seed(42)

<img src="images/arch_si.webp">

**Red neuronal u_net**

In [2]:
def u_net(u_input):
    """
    Red que va a predecir los valores de u
    Args:
        u_input: entrada de la u-net

    Outputs:
        output: la salida de la red    
    """    
    hidden = u_input
    for _ in range(2):
        hidden = tf.keras.layers.Dense(50, activation="tanh")(hidden)
    output = tf.keras.layers.Dense(3)(hidden)
    return output

**Red neuronal auxiliar f_net**

In [4]:
class ParameterLayer(tf.keras.layers.Layer):
    def __init__(self, a, b, trainable=True):
        super(ParameterLayer, self).__init__()
        self._a = tf.convert_to_tensor(a, dtype=tf.float32)
        self._b = tf.convert_to_tensor(b, dtype=tf.float32)
        self.trainable = trainable
    
    def build(self, input_shape):
        self.a = self.add_weight("a", shape=(1,), 
                                 initializer=tf.keras.initializers.Constant(value=self._a),
                                 trainable=self.trainable)
        self.b = self.add_weight("b", shape=(1,), 
                                 initializer=tf.keras.initializers.Constant(value=self._b),
                                 trainable=self.trainable)
        
    def get_config(self):
        return super().get_config()
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

def f_net(f_inputs, a_init=None, b_init=None):
    """
    Red que va a predecir las funciones f
    Args:
        f_inputs: lista de entradas para la f-net
        a_init: valores iniciales de parametro a
        b_init: valores iniciales de parametro b
    
    Outputs:
        output: la salida de la f-net
    """
    hidden = tf.keras.layers.Concatenate()(f_inputs)
    for _ in range(2):
        hidden = tf.keras.layers.Dense(50, activation="tanh")(hidden)
    output = tf.keras.layers.Dense(2)(hidden)
    output = ParameterLayer(a_init, b_init)(output)
    return output

**u-net + f-net = PINN**

In [5]:
def create_PINN(a_init=None, b_init=None, verbose=False):
    """
    Definición de la PINN
    Args:
        a_init: valor inicial de param. a
        b_init: valor inicial de param. b
        verbose: boolean, model summary
    Outputs:
        model: PINN model
    """
    # Input
    t_input = tf.keras.Input(shape=(1,), name="time")

    # u-NN and f-NN
    u = u_net(t_input)
    f = f_net([t_input, u], a_init, b_init)

    # PINN model
    model = tf.keras.models.Model(inputs=t_input, outputs=[u, f])

    if verbose:
        model.summary()
    return model

#### **ED Loss**

<img src="./images/ed_loss.webp">

In [6]:
@tf.function
def ODE_residual_calculator(t, model):
    """ED residual calculate
    
    Args:
    ----
    t: temporal coordinate
    model: PINN model
    
    Outputs:
    --------
    ODE_residual: residual of the governing ODE
    """
    
    # Retrieve parameters
    a = model.layers[-1].a
    b = model.layers[-1].b
    
    with tf.GradientTape() as tape:
        tape.watch(t)
        u, f = model(t)
    
    # Calculate gradients
    dudt = tape.batch_jacobian(u, t)[:, :, 0]
    du1_dt, du2_dt, du3_dt = dudt[:, :1], dudt[:, 1:2], dudt[:, 2:]
    
    # Compute residuals
    res1 = du1_dt - f[:, :1]
    res2 = du2_dt - f[:, 1:]
    res3 = du3_dt - (a*u[:, :1]*u[:, 1:2] + b)
    ODE_residual = tf.concat([res1, res2, res3], axis=1)
    
    return ODE_residual